In [ ]:
%%bash

pip install torchmetrics

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, model_selection, pipeline, preprocessing
import torch
import torchmetrics
from torch import nn, optim, utils

# Building Neural Networks in PyTorch

## Building a linear regression model

In [ ]:
prng = torch.manual_seed(42)
m = 100
features = [
    torch.ones((m, 1)),
    torch.normal(mean=1.0, std=1.0, size=(m, 1), generator=prng)
]
X = torch.cat(features, dim=1)
error = torch.normal(mean=0.0, std=5e-1, size=(m, 1), generator=prng)
beta = torch.tensor([[3.0], [1.5]])
y = X @ beta + error

In [ ]:
_ = plt.plot(X[:, 1], y, 'o')
_ = plt.xlabel(r"$X_1$", fontsize=15)
_ = plt.ylabel("y", fontsize=15, rotation=0)

In [ ]:
prng = torch.manual_seed(42)
_dataset = utils.data.TensorDataset(X,  y)
train_dataset, test_dataset = utils.data.random_split(_dataset, [80, 20], generator=prng)

batch_size = 1
train_dataloader = utils.data.DataLoader(train_dataset, batch_size, shuffle=True)
test_dataloader = utils.data.DataLoader(test_dataset, batch_size)

In [ ]:
# initialize weights
prng = torch.manual_seed(1)
weights = torch.randn((2, 1), generator=prng, requires_grad=True)


def model_fn(X):
    return X @ weights

def loss_fn(y, y_hat):
    return torch.sqrt(torch.mean((y - y_hat)**2))


In [ ]:
learning_rate = 0.001
epochs = 200
log_epochs = 10

for epoch in range(epochs):
    for features, targets in train_dataloader:

        # forward pass
        predictions = model_fn(features)
        loss = loss_fn(targets, predictions)
        
        # backward pass
        loss.backward()
        with torch.no_grad():
            weights -= weights.grad * learning_rate
            weights.grad.zero_()
  
    if epoch % log_epochs == 0:
        print(f'Epoch {epoch}  Loss {loss.item():.4f}')

In [ ]:
print(f'Final Parameters: {weights[:, 0]}')

In [ ]:
with torch.no_grad():
    total_loss = torch.zeros((1,1))
    for features, target in test_dataloader:
        predictions = model_fn(features)
        loss = loss_fn(target, predictions)
        total_loss += loss

print(f"Average test loss: {total_loss.item() / len(test_dataloader)}")

In [ ]:
_ = plt.plot(X[:, 1], y, 'o')
_ = plt.xlabel(r"$X_1$", fontsize=15)
_ = plt.ylabel("y", fontsize=15, rotation=0)

new_features = [
    torch.ones((m, 1)),
    torch.linspace(-2, 4, m).reshape((-1, 1))    
]
X_new = torch.cat(new_features, dim=1)

with torch.no_grad():
    y_new = model_fn(X_new)

_ = plt.plot(X_new[:, 1], y_new)

## Model training usng torch.nn and torch.optim

In [ ]:
loss_fn = nn.MSELoss()

input_size = 2
output_size = 1
model_fn = nn.Linear(input_size, output_size, bias=False)

optimizer = torch.optim.SGD(model_fn.parameters(), lr=learning_rate)

In [ ]:
epochs = 200
log_epochs = 20

for epoch in range(epochs):
    for features, targets in train_dataloader:
        
        # forward pass
        predictions = model_fn(features)        
        loss = loss_fn(predictions, targets)
        
        # backward pass
        loss.backward()        
        optimizer.step()        
        optimizer.zero_grad()    

    if epoch % log_epochs == 0:
        print(f'Epoch {epoch}  Loss {loss.item():.4f}')

In [ ]:
print('Final Parameters:', model_fn.weight)

In [ ]:
y_hat = model_fn(X)

## Building multi-layer perceptrons for classification and regression

### Breast Cancer Classification

In [ ]:
datasets.load_breast_cancer?

In [ ]:
features, targets = datasets.load_breast_cancer(return_X_y=True, as_frame=True)

In [ ]:
features.info()

In [ ]:
features.describe()

In [ ]:
random_state = np.random.RandomState(42)
train_features, test_features, train_targets, test_targets = model_selection.train_test_split(
    features,
    targets,
    test_size=0.1,
    random_state=random_state,
)

In [ ]:
features_preprocessing_pipeline = pipeline.make_pipeline(
    preprocessing.StandardScaler(),
    preprocessing.FunctionTransformer(lambda X: X.astype(np.float32)),
    preprocessing.FunctionTransformer(lambda X: torch.from_numpy(X)),
)

In [ ]:
train_features_tensor = features_preprocessing_pipeline.fit_transform(train_features)
train_targets_tensor = torch.from_numpy(train_targets.to_numpy())

In [ ]:
train_dataset = utils.data.TensorDataset(train_features_tensor, train_targets_tensor)

batch_size = 32
train_dataloader = utils.data.DataLoader(train_dataset, batch_size, shuffle=True)

In [ ]:
loss_fn = nn.NLLLoss()

_, input_size = train_features_tensor.shape
hidden_size = 100
output_size = 2

model_fn = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, output_size),
    nn.Softmax(dim=1)
) 

learning_rate = 1e-2
optimizer = torch.optim.SGD(model_fn.parameters(), lr=learning_rate)

In [ ]:
epochs = 200
log_epochs = 20

for epoch in range(epochs):
    for features, targets in train_dataloader:
        
        # forward pass
        predictions = model_fn(features)        
        loss = loss_fn(predictions, targets)
        
        # backward pass
        loss.backward()        
        optimizer.step()        
        optimizer.zero_grad()    

    if epoch % log_epochs == 0:
        print(f'Epoch {epoch}  Loss {loss.item():.4f}')

### Evaluating the model on the test set

In [ ]:
test_features_tensor = features_preprocessing_pipeline.transform(test_features)
test_targets_tensor = torch.from_numpy(test_targets.to_numpy())

In [ ]:
with torch.no_grad():
    logits = model_fn(test_features_tensor)
    predictions_tensor = logits.argmax(dim=1)

In [ ]:
accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=output_size)
accuracy(predictions_tensor, test_targets_tensor)

### Saving a PyTorch model

In [ ]:
RESULTS_DIR = pathlib.Path("./results")
RESULTS_DIR.mkdir(exist_ok=True)

torch.save(model_fn, RESULTS_DIR / "classifier.ckpt")

### Predicting house prices

In [ ]:
datasets.fetch_california_housing?

In [ ]:
features, targets = datasets.fetch_california_housing(return_X_y=True, as_frame=True)

In [ ]:
features.info()

In [ ]:
features.describe()

In [ ]:
targets.describe() # units are 100k USD

In [ ]:
random_state = np.random.RandomState(42)
train_features, test_features, train_targets, test_targets = model_selection.train_test_split(
    features,
    targets,
    test_size=0.1,
    random_state=random_state,
)

In [ ]:
features_preprocessing_pipeline = pipeline.make_pipeline(
    preprocessing.StandardScaler(),
    preprocessing.FunctionTransformer(lambda X: X.astype(np.float32)),
    preprocessing.FunctionTransformer(lambda X: torch.from_numpy(X)),
)

targets_preprocessing_pipeline = pipeline.make_pipeline(
    preprocessing.FunctionTransformer(lambda X: X.to_numpy()),
    preprocessing.FunctionTransformer(lambda X: X.reshape(-1, 1)),    
    preprocessing.FunctionTransformer(lambda X: X.astype(np.float32)),
    preprocessing.FunctionTransformer(lambda X: torch.from_numpy(X)),
)

In [ ]:
train_features_tensor = features_preprocessing_pipeline.fit_transform(train_features)
train_targets_tensor = targets_preprocessing_pipeline.fit_transform(train_targets)

In [ ]:
train_dataset = utils.data.TensorDataset(train_features_tensor, train_targets_tensor)

batch_size = 32
train_dataloader = utils.data.DataLoader(train_dataset, batch_size, shuffle=True)

In [ ]:
loss_fn = nn.MSELoss()

_, input_size = train_features_tensor.shape
hidden_sizes = [100, 100, 100]
output_size = 1

modules = []
for hidden_size in hidden_sizes:
    module = nn.Linear(input_size, hidden_size)
    modules.append(module)
    modules.append(nn.ReLU())
    input_size = hidden_size
modules.append(nn.Linear(hidden_size, output_size))

model_fn = nn.Sequential(
    *modules
)

learning_rate = 1e-2
optimizer = torch.optim.SGD(model_fn.parameters(), lr=learning_rate)

In [ ]:
epochs = 200
log_epochs = 20

for epoch in range(epochs):
    for features, targets in train_dataloader:
        
        # forward pass
        predictions = model_fn(features)        
        loss = loss_fn(predictions, targets)
        
        # backward pass
        loss.backward()        
        optimizer.step()        
        optimizer.zero_grad()    

    if epoch % log_epochs == 0:
        print(f'Epoch {epoch}  Loss {loss.item():.4f}')

In [ ]:
with torch.no_grad():
    predictions_tensor = model_fn(train_features_tensor)

training_loss = loss_fn(predictions_tensor, train_targets_tensor)
print(f"Training loss: {torch.sqrt(training_loss) * 100_000} USD")

In [ ]:
test_features_tensor = features_preprocessing_pipeline.transform(test_features)
test_targets_tensor = targets_preprocessing_pipeline.transform(test_targets)

In [ ]:
with torch.no_grad():
    predictions_tensor = model_fn(test_features_tensor)

test_loss = loss_fn(predictions_tensor, test_targets_tensor)
print(f"Test loss: {torch.sqrt(test_loss) * 100_000} USD")

In [ ]:
torch.save(model_fn, RESULTS_DIR / "regressor.ckpt")